In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
DATADIR = "positiondata"

In [ ]:
fnames = os.listdir(DATADIR)
case1_cams = [os.path.join(DATADIR, i) for i in fnames if "_1.npy" in i]
case2_cams = [os.path.join(DATADIR, i) for i in fnames if "_2.npy" in i]
case3_cams = [os.path.join(DATADIR, i) for i in fnames if "_3.npy" in i]
case4_cams = [os.path.join(DATADIR, i) for i in fnames if "_4.npy" in i]

In [ ]:
def read_case(cams):
    cam_names = [i.split("_")[0][-1] for i in cams]
    data = [np.load(i) for i in cams]
    return list(zip(cam_names, data))

In [ ]:
def plot_case(case, title):
    assert(len(case) == 3)
    fig, axes = plt.subplots(3, 2, sharex=True, sharey=True)
    fig.set_size_inches(8, 10)
    fig.suptitle(title)
    for i, cam in enumerate(case):
        axes[i, 0].title.set_text("Camera {}, x".format(cam[0]))
        axes[i, 0].set_xlabel("Sample")
        axes[i, 0].set_ylabel("Pixel location")
        axes[i, 0].plot(cam[1][0])
        axes[i, 1].title.set_text("Camera {}, y".format(cam[0]))
        axes[i, 1].set_xlabel("Sample")
        axes[i, 1].set_ylabel("Pixel location")
        axes[i, 1].plot(cam[1][1])

In [ ]:
case_1 = read_case(case1_cams)
case_2 = read_case(case2_cams)
case_3 = read_case(case3_cams)
case_4 = read_case(case4_cams)
plot_case(case_1, "Case 1")
plot_case(case_2, "Case 2")
plot_case(case_3, "Case 3")
plot_case(case_4, "Case 4")

In [ ]:
def shortest_length(case):
    return min([i[1].shape[1] for i in case])

def stack_observations(case):
    last = shortest_length(case)
    arrays = []
    for cam in case:
        x = cam[1][0][:last]
        y = cam[1][1][:last]
        arrays.append(x)
        arrays.append(y)
    return np.stack(tuple(arrays))

In [ ]:
def kutz_plots(case, modes_to_plot=1):
    """
    Make plots like what DR. Kutz does in his book
    """
    u, s, vh = np.linalg.svd(stack_observations(case))
    
    fig = plt.figure(figsize=(8, 14))
    gs = fig.add_gridspec(3, 2)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, :])
    ax4 = fig.add_subplot(gs[2, :])
    
    ax1.stem(s, use_line_collection=True)
    ax1.set_title("Modal Energy")
    ax1.set_xlabel("Mode number")
    ax1.set_ylabel("Energy")
    
    ax2.stem(np.log(s), use_line_collection=True)
    ax2.set_title("Modal Energy, Semilog")
    ax2.set_xlabel("Mode number")
    ax2.set_ylabel("Energy")
    
    fmts = ["-", "--"]
    for i, fmt in zip(range(modes_to_plot), fmts):
        ax3.plot(u[:, i], fmt, label="Mode {}".format(i))
        ax4.plot(vh[i, :], label="Mode {}".format(i))

    ax3.set_title("Modes")
    ax3.set_xlabel("x")
    ax3.set_ylabel("f(x)")
    ax4.set_title("Temporal Behavior of Modes")
    ax4.set_xlabel("Time, samples")
    ax4.set_ylabel("Magnitude")
    ax3.legend()
    ax4.legend()
    
    plt.show()

In [ ]:
kutz_plots(case_1, 2)

In [ ]:
kutz_plots(case_2, 2)

In [ ]:
kutz_plots(case_3, 2)

In [ ]:
kutz_plots(case_4, 2)

In [ ]:
def svd_and_reconstruction(case, modes=None):
    u, s, vh = np.linalg.svd(stack_observations(case))
    if modes is None:
        modes = s.shape[0]    
    s_diag = np.zeros((u.shape[0], vh.shape[0]))
    s_diag[:u.shape[0], :u.shape[0]] = np.diag(s)
        
    return np.matmul(np.matmul(u[:,0:modes], s_diag[0:modes,0:modes]), vh[0:modes, :])

In [ ]:
def plot_reconstructed(case, modes):
    assert(modes>=1)
    x=svd_and_reconstruction(case, modes)
    plt.figure()
    plt.title("Modes: {}".format(modes))
    for i in x:
        plt.plot(i)
        plt.xlabel("Time, samples")
        plt.ylabel("Pixel Position")
        
def plot_reconstructed_subplots(case):
    fig = plt.figure(figsize=(8, 10))
    gs = fig.add_gridspec(3, 2)
    ax = []
    ax.append(fig.add_subplot(gs[0, 0]))
    ax.append(fig.add_subplot(gs[0, 1]))
    ax.append(fig.add_subplot(gs[1, 0]))
    ax.append(fig.add_subplot(gs[1, 1]))
    ax.append(fig.add_subplot(gs[2, 0]))
    ax.append(fig.add_subplot(gs[2, 1]))
    
    for modes in range(1, 7):
        x=svd_and_reconstruction(case, modes)
        for series in x:
            ax[modes-1].plot(series)
            ax[modes-1].set_title("{} modes".format(modes))

In [ ]:
plot_reconstructed_subplots(case_1)

In [ ]:
plot_reconstructed(case_1, 3)

In [ ]:
plot_reconstructed(case_2, 3)

In [ ]:
plot_reconstructed(case_3, 3)

In [ ]:
plot_reconstructed(case_4, 3)